# Packages 

In [1]:
import pandas as pd # pandas package
pd.options.display.max_columns = 30

import numpy as np # numpy package

# matplotlib packages
import matplotlib
import matplotlib.pyplot as plt 
#%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 6.0)

import seaborn as sns # seaborn package

import warnings  # warnings package
warnings.filterwarnings('ignore')

# plotly packages
from chart_studio import plotly
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import iplot

# cufflink packages
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

# interactive shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pathlib import Path # path package
import re #regex package
from textblob import TextBlob #import textblob package

# Load Data set

In [2]:
# load the merged data set into pandas
df = pd.read_csv(Path(r"../Data/Merged Data/merged_data.csv"))

# sample only 5 rows
df.sample(5)

,review_title,review_text,review_star,date,author,page,product
17418,Apple TV,"Great product, easy to use.",5.0 out of 5 stars,"Reviewed in the United States on June 22, 2020",JT,242,Apple TV
11391,Ke veran mas peliculas,Facil de programar,5.0 out of 5 stars,"Reviewed in the United States on January 22, 2021",Gaby sarabia,140,Roku Remote
5530,Great way to cast,This Chromecast is great,5.0 out of 5 stars,"Reviewed in the United States on March 11, 2021",Tony Jay Ong,54,Google ChromeCast
14175,Don't purchase unless you're willing to pay a ...,I wish I had read the reviews before I purchas...,1.0 out of 5 stars,"Reviewed in the United States on October 3, 2020",Susan Ryan,418,Roku Remote
10365,ROKU RULLLLLEEEESSSSSSSS,I have 3 rokus. I ditched cable and only have ...,5.0 out of 5 stars,"Reviewed in the United States on September 2, ...",Pickles,37,Roku Remote


# Pre-cleaning 

1. Remove the rows where `review_text` column were missing
2. Fix `review_star` to have numeric value of ratings 
3. fix `date` to be parsed and converted into datetime object
4. Make suring we have actual string dtype columns
5. Convert `review_title`, `review_text`, `author`, and `product` to string types
6. Adding Sentiment Polarity Scores for `review_text` as a new column `polarity`
7. Adding Length of Reviews as a new column `review_len`
8. Adding the Word Count in each Reviews as a new column `word_count`


## Remove missing values in `review_text`

In [3]:
# original data frame dimensions before removing null values in `review_text`
df.shape

(17740, 7)

In [4]:
# remove null values in `review_text`
df = df[~df['review_text'].isnull()]

# data frame dimensions after removing null values
df.shape

(17696, 7)

## Cleaning `review_star`

In [5]:
## extract rating from strings and convert to float type
df['review_star'] = df['review_star'].apply(lambda x: re.findall(r'^[0-9]\.[0-9]?', x)[0]).astype(float)

# sample only 5 rows
df['review_star'].sample(5)


7971     5.0
6303     5.0
366      5.0
15029    5.0
11525    5.0
Name: review_star, dtype: float64

## Cleaning `date`

In [6]:
# clean date of reviews to parse the actual date and convert to date-time object 
df['date'] = pd.to_datetime(df['date'].apply(lambda x: re.findall(r'(?<=on\ ).*', x)[0])).dt.strftime('%m-%d-%Y')

# sample only 5 rows
df['date'].sample(5)

15618    04-18-2019
4630     03-22-2022
4993     12-21-2021
12394    09-08-2021
17725    06-02-2019
Name: date, dtype: object

## Convert `review_title`, `review_text`, `author`, and `product` to string types

In [7]:
# convert review title column into string type
df['review_title'] = df['review_title'].astype("string")

# convert review text column into string type
df['review_text'] = df['review_text'].astype("string")

# convert author column into string type
df['author'] = df['author'].astype("string")

# convert product column into string type
df['product'] = df['product'].astype("string")

## Adding Sentiment Polarity Scores as `polarity`

In [8]:
# create new column for sentiment polarity scores
df['polarity'] = df['review_text'].map(lambda text: TextBlob(text).sentiment.polarity)

# sample only 5 rows
df['polarity'].sample(5)

1753     0.500000
16208    0.433333
2054    -0.031364
12406    0.000000
9435     0.700000
Name: polarity, dtype: float64

### Preview 5 randomly selected reviews with the highest sentiment polarity score

In [9]:
# Print statement
print('5 random reviews with the highest positive sentiment polarity: \n')

# Randomly sample review texts with highest positive sentiment polarity scores
cl = df.loc[df.polarity == 1, ['review_text']].sample(5).values

# print out each one
for c in cl:
    print(c[0])

5 random reviews with the highest positive sentiment polarity: 

The best option for home
Works great!
Everything works perfect. Can't beat the price. Don't pass up.
Very impressed will buy again
Works great!


### Preview 5 randomly selected reviews with the most neutral sentiment polarity score

In [10]:
# Print statement
print('5 random reviews with the most neutral sentiment(zero) polarity: \n')

# Randomly sample review texts with highest neutral sentiment polarity scores
cl = df.loc[df.polarity == 0, ['review_text']].sample(5).values

# print out each one
for c in cl:
    print(c[0])

5 random reviews with the most neutral sentiment(zero) polarity: 

Ordered this for co-worker
all
Can not get to work
It do what it do
Works as it should.


### Preview 5 randomly selected reviews with the most negative sentiment polarity score

In [11]:
# Print Statement
print('5 reviews with the most negative polarity: \n')

# Randomly sample review texts with highest negative sentiment polarity scores
cl = df.loc[df.polarity <= -1.0, ['review_text']].sample(5).values

# Print out each one
for c in cl:
    print(c[0])

5 reviews with the most negative polarity: 

The interface with all existing services is terrible.
This is a horrible device, although I am an IT Data tech. Cannot get it working after I downloaded 3 apps , gave them my CC card . Now there are fraudulent charges on my account. Please do not purchase this product, you will regret it. !!
Very annoying to use app. Have to pay for monthly fee for app.
Customer service doesn't exist, I bought 3 of these pieces of CRAP for $50.00 and now they are going for $30.00Find some thing else to spend your money on. You can't believe the reviews. Roku will play you for a sap!!
You have to use your phone to control this. Very annoying. Wish I got the fire stick or Roku stick instead


## Adding Length of Reviews as `review_len`

In [12]:
# create new column called review_len for length of reviews
df['review_len'] = df['review_text'].astype(str).apply(len)

# sample only 5 rows
df['review_len'].sample(5)

8584      27
11292    109
2672      71
600      192
16514     30
Name: review_len, dtype: int64

## Adding the Word Count in each Reviews as `word_count`

In [13]:
# create new column called word count for number of words in each reviews
df['word_count'] = df['review_text'].apply(lambda x: len(str(x).split()))

# sample only 5 rows
df['word_count'].sample(5)

626      25
11563    26
11708     7
10419    40
11221    22
Name: word_count, dtype: int64

## Re-examine data now

In [14]:
# preview new clean columns with only 10 samples
df.sample(10)

,review_title,review_text,review_star,date,author,page,product,polarity,review_len,word_count
10278,Don’t hesitate!,I love this! I finally cut my cord and bought ...,5.0,02-14-2020,Karen Baker,28,Roku Remote,0.246168,699,143
8305,Need More faster,Catching next program freezes.,5.0,12-25-2019,emon,331,Google ChromeCast,0.300000,30,4
1192,This is in our guest room and there happy I do...,Great product happy with the purchase will def...,5.0,01-04-2022,ATIZZY,120,Amazon Fire TV Stick,0.533333,64,10
14411,New Roku,"Works great, Alexa seems to mind us. can turn ...",5.0,11-10-2020,Pawsie13,442,Roku Remote,0.166032,228,48
11921,Must reset remote repeatedly,I disliked the remote control.,5.0,01-01-2022,L,193,Roku Remote,-0.150000,30,5
2380,Love it,I’ve used Firestick for 3 yrs. One for every TV,5.0,05-11-2022,Katherine Schloemer,239,Amazon Fire TV Stick,0.000000,47,10
7883,It's not perfect,Instructions werent great I had to call for te...,3.0,09-07-2019,2ez,289,Google ChromeCast,0.250000,145,27
222,Works very well and actually saves money from ...,"Works very well, has great 4K video quality th...",5.0,02-08-2022,BHaskell,23,Amazon Fire TV Stick,0.533333,276,54
15196,A nice upgrade to my 4th gen Apple TV. 4k is g...,Bought a 4k tv so I decided to buy this as wel...,5.0,05-12-2019,Darick Petrik,20,Apple TV,0.531061,170,36
13500,Pretty close to worthless,Roku RF remotes are the worst remotes I've eve...,1.0,01-27-2020,Thomas in Carpentersville,351,Roku Remote,-0.088235,877,159


# Exploratory Data Analysis 

1. Examine a five-number summary of the numerical and categorical columns
2. Checking for Missing Data
3. Univariate Analysis
    
    -  Distribution of review sentiment polarity score
    -  Distribution of review ratings
    - Distribution of review lengths
    - Distribution of review word counts
    -  Distribution of products


## 1. Examine a five-number summary of the numerical and categorical columns

In [15]:
# 5 number summary of the numerical columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
review_star,17696.0,4.479204,1.185062,1.0,5.000000,5.00,5.0,5.0
page,17696.0,232.748361,142.237177,1.0,111.000000,222.00,352.0,500.0
polarity,17696.0,0.299442,0.309198,-1.0,0.025666,0.26,0.5,1.0
review_len,17696.0,205.686370,337.735191,1.0,34.000000,91.00,248.0,10046.0
word_count,17696.0,38.499491,62.225447,1.0,6.000000,17.00,47.0,1766.0


**Interpretation**


1. `review_star`: The range of the review ratings is 1 to 5 stars where the mean across all brands for streaming devices from this data set is 4.47.

2. `page`: The page number does not tell us much aside from that fact that we were able to scrape customer reviews from pages 1 to pages 500.

3. `polarity`: This is an interesting summary because the polarity sentiment score tells us on average we are seeing a 0.299 where -1.0 is the lowest and 1.0 is the highest. So typically on average, we are a bit past the neutral sentiment when looking at reviews on average.

4. `review_len`: This column specifies the length of each review, in this case on average we are seeing 205.68 characters per reviews with the least being 34 and max being 10,000 characters.

5. `word_count`: Where as the word count counts only words and on average there are 38.5 words per review with the least being 1 word and the most being 1,766 words.


**Looking at Categorical Columns**:

1. product

In [16]:
# summary statistic of products
df[['product']].describe().T

,count,unique,top,freq
product,17696,4,Roku Remote,5000


**Interpretation**

1. `product`: Since not all reviews were scraped equally from each brand and from our pre-cleaning work we also had to drop several rows with missing reviews, our new summary statistics of the `product` column tells us that `Roku Remote` product has the most reviews at 5000.

## 2. Checking for Missing Data


In [17]:
# check for total missing rows across all columns
df.isna().sum()

review_title    3
review_text     0
review_star     0
date            0
author          1
page            0
product         0
polarity        0
review_len      0
word_count      0
dtype: int64

**Interpretation**: 

1. Since the columns `review_title` and `author` contain very little missing values and also does not impact the modeling phase later on, we will still keep these rows

## 3. Univariate Analysis

- Distribution of review sentiment polarity score
- Distribution of review ratings
- Distribution of review lengths
- Distribution of review word counts
- Distribution of products

### Distribution of review sentiment polarity score

In [18]:
# plot review sentiment polarity scores with plotly
df['polarity'].iplot(
    kind='hist', # histogram
    bins = 50, # number of bins
    xTitle='Polarity', # xlabel
    color = "blue", # bar chart color
    linecolor='black', # bar chart line color
    yTitle='Count', # y label
    title='Sentiment Polarity Distribution') # title

**Interpretation**

1. The distribution seems to be left skewed with a longer left tail and the rest of the distribution is clustered around the positive polarity sentiment scores with a noticable peak at 0 or neutral.

# Distribution of review ratings


In [19]:
# create a data frame to count the number of customer reviews by products and also rename a column as count
rating_counts = pd.DataFrame(df[['review_star']].value_counts()).reset_index().rename(columns={'review_star': 'Ratings', 0:'count'})
rating_counts

,Ratings,count
0,5.0,14206
1,1.0,1309
2,4.0,946
3,3.0,671
4,2.0,564


In [20]:
# use plotly to create a bar graph of number of reviews by brand and using customized color coding 
fig = px.bar(rating_counts, x="Ratings", y="count", color="Ratings", title= "Customer Ratings", text_auto = '.2s')

# figure customization
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False,
 marker_line_width=1.5)


**Interpretation**

1. As expected, ratings are dominated by the most satisfied customers who wants to leave the highest rating of 5/5 stars and very dissatisfied customers leaving poor ratings 1/5 and 2/5.

# Distribution of review lengths


In [21]:
# plot review lengths plotly
df['review_len'].iplot(
    kind='hist', # histogram
    bins = 200, # number of bins
    xTitle='Length', # xlabel
    color = "green", # bar chart color
    linecolor='black', # bar chart line color
    yTitle='Count', # y label
    title='Length of Reviews Distribution') # title

**Interpretation** 

1. A very right-skewed distribution with most reviews falling under 200 characters or less for the most counts.

# Distribution of review word counts

In [22]:
# plot review lengths plotly
df['word_count'].iplot(
    kind='hist', # histogram
    bins = 200, # number of bins
    xTitle='Words', # xlabel
    color = "orange", # bar chart color
    linecolor='black', # bar chart line color
    yTitle='Count', # y label
    title='Number of Words in Reviews Distribution') # title

**Interpretation**

1. A very right-skewed distribution with most reviews falling under 30 words or less per review for the most counts.

# Distribution of products

In [23]:
# create a data frame to count the number of customer reviews by products and also rename a column as count
product_counts = pd.DataFrame(df[['product']].value_counts()).reset_index().rename(columns={0:'count'})
product_counts

,product,count
0,Roku Remote,5000
1,Google ChromeCast,4997
2,Amazon Fire TV Stick,4959
3,Apple TV,2740


In [24]:
# use plotly to create a bar graph of number of reviews by brand and using customized color coding 
fig = px.bar(product_counts, x="product", y="count", color="product", title= "Streaming Devices by Brands",
            color_discrete_map={'Roku Remote':'purple', 'Google ChromeCast': 'red', 'Amazon Fire TV Stick':'orange', 'Apple TV':'black'},
            text_auto = '.2s')

# figure customization
fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False,
marker_line_width=1.5, opacity=0.75)

**Interpretation**

1. Originally, Apple TV was only able to scrape 2,740 reviews and this seems to still be the case.
2. However, since we removed rows where reviews were missing, we can check accordingly and see that 3 rows were missing reviews from Google ChromeCast and 41 rows were missing reviews from Amazon Fire TV Stick.